In [2]:
# necessary Modules

import random
from hashlib import sha256, sha3_256

In [44]:
# function to create s (secret key) and v (public key) with given g (generator) and p (prime mod)  
def generate_keys(g, p):
    # secret key s
    s = random.randint(1, p-2)
    # public key = g^s mod p
    v = pow(g, s, p)
    return s, v


# function to generate n random secret and public keys
def generate_n_random_keys(n):
    # a list to store all the pubkey and secrect keys
    keys_list = []
    for _ in range(n):
        # generate random keys
        x_temp, y_temp = generate_keys(g, p)
        # append random keys
        keys_list.append([x_temp, y_temp])
    return keys_list

# function to concat the real key to the random key list and shuffle
def concat_and_shuffle_total_keys(keys_list, x, y):
    # append the real key
    keys_list.append([x, y])
    # do the shuffle
    random.shuffle(keys_list)
    return keys_list

# function to get the real public key's corresponding position
def get_real_key_position(keys_list, y):
    # make the public keys into one list for index
    public_key_list = [key[1] for key in keys_list]
    # return the real public key index
    return public_key_list.index(y)

# function to get the Y, which is the string containing all the public key used in this signature
def get_Y(keys_list):
    # make the public keys into one list for getting the Y
    public_key_list = [key[1] for key in keys_list]
    # generate Y, which is the string containing all the public key used in this signature
    Y = ''.join([str(i) for i in public_key_list])
    return Y

# function to get the initial random value r and R used in the signature
def get_initial_random_r_and_R(g, p):
    r, R = generate_keys(g, p)
    return r, R

# function to generate the signature
def sign(msg, Y, g, r, p, keys_list, x_real, y_real):
    # find the real pubkey's position and +1 to get the next position
    idx = (get_real_key_position(total_keys_list, y_real) + 1) % len(total_keys_list)
    # initial the e_list with None (which represents c in the thesis)
    e_list = [None for _ in range(len(keys_list))]
    # initial the signature list with None 
    sig_list = [None for _ in range(len(keys_list))]
    # calculate the h with Y and using another hash function (thesis 4.1.1)
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16)
    # calculate y wave (thesis 4.1.1)
    y_wave = pow(h, x_real, p)
    # calculate the first e with the random number "r" (thesis 4.1.2)
    e_initial = int(sha256((str(Y) + str(y_wave) +str(msg) + str(pow(g,r,p)) + str(pow(h,r,p))).encode()).hexdigest(), 16)
    # place the "e" into e_list
    e_list[idx] = e_initial
    # do the while loop, in order to calculate each position's value until the real public key's position
    while keys_list[idx][1] != y_real:
        # find the "e" used in last term loop
        e_last_one = e_list[idx]
        # generate the random value for the signature (thesis 4.1.3 start)
        _, random_sig = generate_keys(g, p)
        # place it to the position
        sig_list[idx] = random_sig
        # find this term cooresponding y
        temp_y = keys_list[idx][1]
        # do the calculation (thesis 4.1.3 for last two parts)
        # gy part
        calculation = (pow(g, random_sig, p)*pow(temp_y, e_last_one, p)) % p
        # hy part
        calculation_1 = (pow(h, random_sig, p)*pow(y_wave, e_last_one, p)) % p
        # calculate the e for this term (thesis 4.1.3 end)
        temp_e = int(sha256((str(Y) + str(y_wave) +str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16)
        # index += 1
        idx = (idx + 1) % len(total_keys_list)
        # place the "e"
        e_list[idx] = temp_e
        
    # q is the order for p, see paper page 6, part 4, the first sentence. 
    q = p - 1
    # do the real signature (thesis 4.1.4)
    s_real = (r - x_real * e_list[idx]) % q
    sig_list[idx] = s_real
    public_key_list = [key[1] for key in keys_list] 
    total_res = {
        'e1': e_list[0],
        'sig_list': sig_list,
        'public_key_list': public_key_list,
        'tag': y_wave,
        'msg': msg,
        'e_list': e_list
    }
    return total_res
        
    
    

# define the prime p, and the g (primitive root of p) used in the whole process
p = 1000000007  # prime
g = 5  # primitive root of p

msg = 'Hello, Ring Signature'
random_keys_count = 10

random_keys_list = generate_n_random_keys(random_keys_count)
x_real, y_real = generate_keys(g, p)
total_keys_list = concat_and_shuffle_total_keys(random_keys_list, x_real, y_real)
real_key_index = get_real_key_position(total_keys_list, y_real)
Y = get_Y(total_keys_list)
r, R = get_initial_random_r_and_R(g, p)
total_res = sign(msg, Y, g, r, p, total_keys_list, x_real, y_real)

display(total_res)



{'e1': 99811632369012577900418597917686451392096661144658592216238842444238589791538,
 'sig_list': [459933213,
  531241289,
  525215880,
  283936388,
  420685294,
  836337457,
  115347143,
  258388225,
  355300820,
  141760805,
  408588480],
 'public_key_list': [710012029,
  4466106,
  181786826,
  201773906,
  724239459,
  974435443,
  835333083,
  598633250,
  411776632,
  693709908,
  858631412],
 'tag': 785525098,
 'msg': 'Hello, Ring Signature',
 'e_list': [99811632369012577900418597917686451392096661144658592216238842444238589791538,
  19443073889487748989700997757812565579995193510718558869102097508794082243722,
  37012645531509120735203414257124774037510371072148082024819237312111321922955,
  671257197927806357962035969250208406100853957851648570657033530731190088855,
  104824617443239296942158187902492442119841987191577801683534597311130496196826,
  37435431453230116543904754715512821864529284961942263476445923588774247640874,
  656765624495354112860805544299988054380028551519

In [4]:
def verify(total_res):
    public_key_list = total_res['public_key_list']
    Y = ''.join([str(i) for i in public_key_list])
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16)

    sig_list = total_res['sig_list']
    temp_e_V = e1 = total_res['e1']
    msg = total_res['msg']
    tag = total_res['tag']
    
    for i in range(len(sig_list)):
        print('temp_e_V:',  temp_e_V)
        temp_sig = sig_list[i]
        temp_y = public_key_list[i]
        calculation = (pow(g, temp_sig, p)*pow(temp_y, temp_e_V, p)) % p
        calculation_1 = (pow(h, temp_sig, p)*pow(tag, temp_e_V, p)) % p
        temp_e_V = int(sha256((str(Y) + str(tag) + str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16)
    
    print(e1)
    print(temp_e_V)
    print(temp_e_V == e1) 

verify(total_res)

temp_e_V: 47111303927130558371729202037086723609932139677933603935963851617944420835686
temp_e_V: 98225134333774521677269740447249684453883327501947092157798963006837519446155
temp_e_V: 71163050164265827047854608145740619072694770971401609490095885158234486116155
temp_e_V: 77170230064583136480455732443551177704183068917384738918516557632547881125812
temp_e_V: 47581385473974822091619754424202616680142119156390870524548256151342697072179
temp_e_V: 113341114984676806171576405939798474921420151149376647905692516415806357467536
temp_e_V: 77910994487592688885464025510646805495996920242336966218501569556499244165209
temp_e_V: 92110003357510575642963321687827121771941207226416349213804344240693680159921
temp_e_V: 50622445171916285881972943432983314467732691863599672835220915150151542681638
temp_e_V: 18620763527331296864989265593002966777422532423587311852815802832850237067733
temp_e_V: 26392061964920997030007175765747467452223723052256421803717813530463628570472
4711130392713055837172920203708

In [46]:
def check_tag(signature_res, signature_res_1):
    return signature_res['tag'] == signature_res_1['tag']

msg_1 = 'new test'
total_res_1 = sign(msg_1, Y, g, r, p, total_keys_list, x_real, y_real)
display(total_res_1)
print("total_res_1['tag'] == total_res['tag']:",check_tag(total_res, total_res_1))

{'e1': 91354265423594592018245142758955353852589577771333153785324809116654326139287,
 'sig_list': [639961771,
  432149570,
  434350803,
  832925190,
  559439030,
  255147491,
  27016779,
  183306952,
  656091192,
  887993090,
  992849301],
 'public_key_list': [710012029,
  4466106,
  181786826,
  201773906,
  724239459,
  974435443,
  835333083,
  598633250,
  411776632,
  693709908,
  858631412],
 'tag': 785525098,
 'msg': 'new test',
 'e_list': [91354265423594592018245142758955353852589577771333153785324809116654326139287,
  86554258523389404772238651992605756847632861061293184655652637769077738802483,
  7549885009873618093610923532258981963843714368303494387947910040621525371120,
  5727023709440709954248882614697904939457133004440735560800199260541389285239,
  20127984675419090306096445430610249855258423397655805275945094625286544456274,
  76264978385907387235714060915363626260489663184214801104896249985713587844601,
  312161643951135016696006619719336959018833969072655143839472374

total_res_1['tag'] == total_res['tag']: True


In [6]:
def is_primitive_root(p, g):
    n = p - 1
    factors = []
    i = 2
    while i * i <= n:
        if n % i == 0:
            factors.append(i)
            while n % i == 0:
                n //= i
        i += 1
    if n > 1:
        factors.append(n)

    for factor in factors:
        if pow(g, (p-1) // factor, p) == 1:
            return False
    return True

p = 1000000007 
g = 5 

result = is_primitive_root(p, g)
print(f"g = {g} is a primitive root of p = {p}: {result}")


g = 5 is a primitive root of p = 1000000007: True
